<a href="https://colab.research.google.com/github/Pavel184/PyTorch_basic_course/blob/Homework_lesson_3/Homework_lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework_lesson_3

# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import torch
from torchvision import transforms, datasets
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_3/train.csv')
data_train.shape

(1503424, 18)

In [4]:
# Столбцы для ohe
onehot_encoder = OneHotEncoder(sparse=False)
cat_columns = ['user_type','parent_category_name']
# Столбцы для обработки пропусков
NUM_FEATURE_NAMES = ['price', 'item_seq_number', 'image_top_1']
CAT_FEATURE_NAMES = ['activation_date', 'user_type', 'parent_category_name']

In [5]:
def preproc_test_train(df):
  # Обработка пропусков
  for i in CAT_FEATURE_NAMES:
      column_mode = df[i].mode()[0]
      df[i] = df[i].fillna(column_mode)
    
  for i in NUM_FEATURE_NAMES:
      column_median = df[i].median()
      df[i] = df[i].fillna(column_median)
  # Генерация новых признаков из даты активации
  df['activation_date'] = pd.to_datetime(df['activation_date'])
  df['act_date_dayofweek'] = [d.dayofweek for d in df["activation_date"]] 
  df['act_date_dayofyear'] = [d.dayofyear for d in df["activation_date"]]
  #df['act_date_year'] = [d.year for d in df["activation_date"]] # Получается константный признак
  df['act_date_month'] = [d.month for d in df["activation_date"]]
  df['time_delta'] = (datetime.strptime('2016/12/31', "%Y/%m/%d") - df['activation_date']).dt.days.abs()
  # Перевод id в десятеричную систему
  df['item_id'] = df['item_id'].apply(int, base=16)
  df['user_id'] = df['user_id'].apply(int, base=16)
  # OHE
  encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(df[cat_columns])).astype(int)
  df = pd.concat([df, encoded_categorical_columns], axis=1)
  # Генерация новых признаков
  df = df.merge((df.groupby(by=['region','title']).agg('price').sum()/df.groupby(by=['region','title']).agg('category_name').count()).rename('aver_price_per_reg_per_title'), how='left',on=['region','title'])
  df = df.merge((df.groupby(by=['city','title']).agg('price').sum()/df.groupby(by=['city','title']).agg('category_name').count()).rename('aver_price_per_city_per_title'), how='left',on=['city','title'])
  return df

data_train = preproc_test_train(data_train)

In [6]:
data_train.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,act_date_dayofweek,act_date_dayofyear,act_date_month,time_delta,0,1,2,3,4,5,6,7,8,9,10,11,aver_price_per_reg_per_title,aver_price_per_city_per_title
0,203490245125805,246357444193017,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,1,87,3,87,0,1,0,0,0,0,0,1,0,0,0,0,400.000000,400.000000
1,50216779673981,63422516363287,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,6,85,3,85,0,1,0,0,0,1,0,0,0,0,0,0,3000.000000,3000.000000
2,205074739148252,160404018681571,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,0,79,3,79,0,1,0,1,0,0,0,0,0,0,0,0,4000.000000,4000.000000
3,2858017477354,210405295806253,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,5,84,3,84,1,0,0,0,0,0,0,1,0,0,0,0,1854.347826,1775.000000
4,136961110495915,263129098025995,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,3,75,3,75,0,1,0,0,0,0,0,0,0,1,0,0,70950.000000,63144.444444


In [8]:
# Выделение столбцов для тренировочного датасета
columns = ['price', 'item_seq_number', 'image_top_1', 'act_date_dayofweek',  
          'act_date_dayofyear', 'act_date_month', 'time_delta', 
           'aver_price_per_reg_per_title', 'aver_price_per_city_per_title',
           0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 'deal_probability']
target = ['deal_probability']

In [9]:
# Создание тренировочного датасета
df = pd. DataFrame(data_train[columns])
df = df.apply(pd.to_numeric)
df.head(5)

,price,item_seq_number,image_top_1,act_date_dayofweek,act_date_dayofyear,act_date_month,time_delta,aver_price_per_reg_per_title,aver_price_per_city_per_title,0,1,2,3,4,5,6,7,8,9,10,11,deal_probability
0,400.0,2,1008.0,1,87,3,87,400.000000,400.000000,0,1,0,0,0,0,0,1,0,0,0,0,0.12789
1,3000.0,19,692.0,6,85,3,85,3000.000000,3000.000000,0,1,0,0,0,1,0,0,0,0,0,0,0.00000
2,4000.0,9,3032.0,0,79,3,79,4000.000000,4000.000000,0,1,0,1,0,0,0,0,0,0,0,0,0.43177
3,2200.0,286,796.0,5,84,3,84,1854.347826,1775.000000,1,0,0,0,0,0,0,1,0,0,0,0,0.80323
4,40000.0,3,2264.0,3,75,3,75,70950.000000,63144.444444,0,1,0,0,0,0,0,0,0,1,0,0,0.20797


In [10]:
# Перевод во float32
df = df.astype('float32')

In [11]:
# Разбивка на train test
train, test = train_test_split(df, random_state=13, test_size = 0.25)

In [12]:
# Нормализация
std = StandardScaler()

In [13]:
cols_std =['price', 'item_seq_number', 'image_top_1', 'act_date_dayofweek', 
          'act_date_dayofyear', 'act_date_month', 'time_delta', 
           'aver_price_per_reg_per_title', 'aver_price_per_city_per_title',
          0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [14]:
train[cols_std] = std.fit_transform(train[cols_std])
test[cols_std] = std.transform(test[cols_std])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.



In [15]:
train.head(3)

,price,item_seq_number,image_top_1,act_date_dayofweek,act_date_dayofyear,act_date_month,time_delta,aver_price_per_reg_per_title,aver_price_per_city_per_title,0,1,2,3,4,5,6,7,8,9,10,11,deal_probability
31723,-0.062684,-0.131918,-0.183251,0.028151,0.357536,-0.002307,0.357536,-0.031698,-0.013604,-0.5486,0.630845,-0.237365,-0.360501,-0.110163,-0.367004,-0.190318,-0.931255,-0.336632,4.222129,-0.211774,-0.245998,0.0000
929954,-0.062684,-0.130126,-0.886306,-0.956525,1.593410,-0.002307,1.593410,-0.031709,-0.013610,-0.5486,0.630845,-0.237365,-0.360501,-0.110163,-0.367004,-0.190318,1.073819,-0.336632,-0.236847,-0.211774,-0.245998,0.0000
1143227,-0.026708,-0.132814,-0.184321,0.520490,0.604711,-0.002307,0.604711,-0.014959,-0.005161,-0.5486,0.630845,-0.237365,-0.360501,-0.110163,-0.367004,-0.190318,-0.931255,-0.336632,4.222129,-0.211774,-0.245998,0.7376


In [16]:
# Подготовка датасета для подачи в модель
class MyOwnDS(torch.utils.data.Dataset):
   
    def __init__(self, _init_dataset, transform=None):
        self._x = _init_dataset.drop(target, axis = 1).values.astype(np.float32)
        self._Y = _init_dataset[target].values.astype(np.float32)

    def __len__(self):
        return len(self._x)

    def __getitem__(self, idx):
        x = self._x[idx]
        x = torch.tensor(x)
        Y = self._Y[idx]
        Y = torch.tensor(Y)
        return x, Y
    

train_dataset = MyOwnDS(train)
test_dataset = MyOwnDS(test)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=5120,
                          shuffle=True,
                          num_workers=1)
val_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=5120,
                          shuffle=True,
                          num_workers=1)

In [17]:
# Архитектура сети
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return torch.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "sigmoid")
        
    def forward(self, x):
        #x = self.bn1(x) # В данной задаче более уместна нормализация всего датасета. Нормализация по бачам имеет смысл когда данные разделены между собой(картинки) или в задачах nlp, когда предложения могут иметь паденговые токены(разную длину) и "вертикальная" нормализация неприменима
        x = self.fc1(x)
        #x = self.dp(x)
        x = self.fc2(x)
        return x

In [18]:
# Функция потерь
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

Adam

In [19]:
# Инициализация
net = FeedForward(21, 100)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = RMSLELoss()

In [20]:
# Обучение модели
hist_train_1 = []
hist_val_1 = []
for epoch in tqdm(range(50)):  
    net.train()
    loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, target = data[0], data[1]
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    hist_train_1.append(loss_train)

    net.eval()
    loss_val = 0
    for x, Y in val_loader:
        output = net(x)
        loss = criterion(output, Y)
        loss_val += loss.item()
    print(f"Epoch {epoch} valid_loss {loss_val}")
    hist_val_1.append(loss_val)

print('Training is finished!')

  2%|▏         | 1/50 [00:24<20:02, 24.53s/it]

Epoch 0 valid_loss 13.416602462530136


  4%|▍         | 2/50 [00:48<19:32, 24.42s/it]

Epoch 1 valid_loss 13.377610966563225


  6%|▌         | 3/50 [01:13<19:10, 24.48s/it]

...

Epoch 47 valid_loss 13.257458716630936


 98%|█████████▊| 49/50 [19:50<00:24, 24.40s/it]

Epoch 48 valid_loss 13.261410415172577


100%|██████████| 50/50 [20:15<00:00, 24.30s/it]

Epoch 49 valid_loss 13.253108695149422
Training is finished!


Val_loss

In [21]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_val_1,
                    name='Val_loss'))
fig.show()

Train_loss

In [22]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_train_1,
                    name='Train_loss'))
fig.show()

RMSprop

In [23]:
net = FeedForward(21, 100)

optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)
criterion = RMSLELoss()

In [24]:
hist_train_2 = []
hist_val_2 = []
for epoch in tqdm(range(50)):  
    net.train()
    loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, target = data[0], data[1]
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    hist_train_2.append(loss_train)

    net.eval()
    loss_val = 0
    for x, Y in val_loader:
        output = net(x)
        loss = criterion(output, Y)
        loss_val += loss.item()
    print(f"Epoch {epoch} valid_loss {loss_val}")
    hist_val_2.append(loss_val)

print('Training is finished!')

  2%|▏         | 1/50 [00:25<20:52, 25.55s/it]

Epoch 0 valid_loss 13.479054808616638


  4%|▍         | 2/50 [00:51<20:33, 25.70s/it]

Epoch 1 valid_loss 13.42818370461464


  6%|▌         | 3/50 [01:17<20:08, 25.71s/it]

...

Epoch 47 valid_loss 13.33661487698555


 98%|█████████▊| 49/50 [21:03<00:25, 25.81s/it]

Epoch 48 valid_loss 13.324292317032814


100%|██████████| 50/50 [21:29<00:00, 25.79s/it]

Epoch 49 valid_loss 13.318510219454765
Training is finished!


Val_loss

In [25]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_val_2,
                    name='Val_loss'))
fig.show()

Train_loss

In [26]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_train_2,
                    name='Train_loss'))
fig.show()

SGD

In [27]:
net = FeedForward(21, 100)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = RMSLELoss()

In [28]:
hist_train_3 = []
hist_val_3 = []
for epoch in tqdm(range(50)):  
    net.train()
    loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, target = data[0], data[1]
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    hist_train_3.append(loss_train)

    net.eval()
    loss_val = 0
    for x, Y in val_loader:
        output = net(x)
        loss = criterion(output, Y)
        loss_val += loss.item()
    print(f"Epoch {epoch} valid_loss {loss_val}")
    hist_val_3.append(loss_val)

print('Training is finished!')

  2%|▏         | 1/50 [00:25<21:10, 25.92s/it]

Epoch 0 valid_loss 16.594731256365776


  4%|▍         | 2/50 [00:51<20:38, 25.80s/it]

Epoch 1 valid_loss 14.780807614326477


  6%|▌         | 3/50 [01:17<20:12, 25.79s/it]

...

Epoch 47 valid_loss 13.51503573358059


 98%|█████████▊| 49/50 [20:57<00:25, 25.65s/it]

Epoch 48 valid_loss 13.509246706962585


100%|██████████| 50/50 [21:22<00:00, 25.66s/it]

Epoch 49 valid_loss 13.510991886258125
Training is finished!


Val_loss

In [29]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_val_3,
                    name='Val_loss'))
fig.show()

Train_loss

In [30]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist_train_3,
                    name='Train_loss'))
fig.show()

Из всех оптимизаторов, лучше всех показал себя Adam